## Pooling Cross-Section Across Time: Simple Panel Data Methods

Pooled cross sections consist of random samples from the same population at differetn points in time. In this notebook, we first introduce this type of data set and how to use it for estimating changes over time. Then, we cover difference-in-differences estimators, an important application of pooled cross-sections for identifying causal effects.

Panel data resemble pooled cross sectional data in that we have observations at different points in time. The key difference is that we observe the same *cross-sectional* units, for example individuals or firms. Panel data methods require the data to be organized in a systematic way, as discussed in later sections. Finally, we introduce the first panel data method, first differenced estimation.

**Topics**
1. Pooled Cross-Sections
2. Difference-in-Differences
3. Organizing Panel Data
4. First Differenced Estimator

### 1. Pooled Cross-Sections

If we have random samples at different points in time, this does not only increase the overall sample size and thereby the statistical precision of our analyses. It also allows to study changes over time and shed addtional light on relationships between variables.

#### Wooldridge, Example 13.2: Changes to the Return to Education and the Gender Wage Gap

The data set *cps78_85* includes two pooled cross-sections for years 1987 and 1985. The dummy variable *y85* is equal to one for observations in 1985 and to zero for 1978. We estimate a model for the log wage *lwage* of the form

$$lwage = \beta_0 + \delta_0 y85 + \beta_1 educ + \delta_1(y_85 \cdot educ) + \beta_2 exper + \beta_3 \frac{exper^2}{100} + \beta_4 union + \beta_5 female + \delta_5 (y85 \cdot female) + u$$

Note that we divide $exper^2$ by 100 and thereby multiply $\beta_3$ by 100 compared to the results reported in Wooldridge (2019). The parameter $\beta_1$ measures the return to education in 1978 and $\delta_1$ is the *difference* of the return to education in 1985 related to 1978. Likewise, $\beta_5$ is the gender wage gap in 1978 and $\delta_5$ is the change of the wage gap.

From the estimated model, the return to education is estimated to have increased by $\hat{\delta_1}$ = 0.0185 and the gender gap decreased in absolute value from $\hat{\beta_5}$ = -0.3167 to $\hat{\beta_5} - \hat{\delta_5}$ = -0.2316, even though this change is only marginally significant.

In [1]:
# Import modules
import wooldridge as woo
import pandas as pd
import statsmodels.formula.api as smf

In [2]:
# Import the 'cps78_85' data set
cps78_85 = woo.dataWoo('cps78_85')

In [3]:
# OLS reuslts including interaction terms
reg = smf.ols(formula = 'lwage ~ y85*(educ+female) + exper +'
             'I((exper**2)/100) + union', data = cps78_85)
results = reg.fit()

In [4]:
# Print regression table
table = pd.DataFrame({'b': round(results.params, 4),
                     'se': round(results.bse, 4),
                     't': round(results.tvalues, 4),
                     'pval': round(results.pvalues, 4)})
print(f'Regression Table: \n{table}\n')

Regression Table: 
                            b      se        t    pval
Intercept              0.4589  0.0934   4.9111  0.0000
y85                    0.1178  0.1238   0.9517  0.3415
educ                   0.0747  0.0067  11.1917  0.0000
female                -0.3167  0.0366  -8.6482  0.0000
y85:educ               0.0185  0.0094   1.9735  0.0487
y85:female             0.0851  0.0513   1.6576  0.0977
exper                  0.0296  0.0036   8.2932  0.0000
I((exper ** 2) / 100) -0.0399  0.0078  -5.1513  0.0000
union                  0.2021  0.0303   6.6722  0.0000



### 2. Difference-in-Differences

Wooldridge (2019, Section 13.2) discusses an important type of application for pooled cross-sections. **Difference-in-differences (DID)** estimators estimate the effect of a policy intervention (in the broadest sense) by comparing the chagne over time of an outcome of interest between an affected and an unaffected group of observations.

In a regrssion framework, we regress the outcome of interest on a dummy variable for the affected ("**treatment**") group, a dummy indicating observations after the treatment and an interaction term between both. The coefficient fo this interaction term can then be a good estimator for the effect of interest, controlling for initial differences between the groups and contemporaneous changes over time.

#### Wooldridge, Example 13.3: Effect of a Garbage Incinerator's Location on Housing Prices

We are inerested in whether and how much the construction of a new garbage incinerator affected the value of nearby houses. We use the data set *KIELMC* to first estimate separate models for 1978 (before there were any rumors about the new incinerator) and 1981 (when the construction began). In 1981, the houses close to the construction site were cheaper by an average of \\$30,688.26. But this was not only due to the new incinerator since even in 1978, nearby houses were cheaper by an average of \\$18,824.37. The difference of these differentces $\hat{\delta}$ = \\$30,688.27 - \\$18,824.37 = \\$11,863.90 is the **DID** estimator and is arguably a better indicator of the actual effect.

The **DID** estimator can be obtained mroe conveniently using a joint regression model with the interaction term as described above. The estimator $\hat{\delta}$ = \$11,863.90 can be directly seen as the coefficient of the interaction term. Conveniently, standard regression tables include *t* tests of the hypothesis that the actual effect is equal to zero. For a one-sided test, the *p* value is $\frac{1}{2} \cdot$ 0.113 = 0.056, so there is some statistical evidence of a negative impact.

In [5]:
# Import modules
import wooldridge as woo
import pandas as pd
import statsmodels.formula.api as smf

In [6]:
# Import the 'kielmc' data set
kielmc = woo.dataWoo('kielmc')

In [7]:
# Separate regression for 1978 and 1981
y78 = (kielmc['year'] == 1978)
reg78 = smf.ols(formula = 'rprice ~ nearinc', data = kielmc, subset = y78)
results78 = reg78.fit()

y81 = (kielmc['year'] == 1981)
reg81 = smf.ols(formula = 'rprice ~ nearinc', data = kielmc, subset = y81)
results81 = reg81.fit()

In [8]:
# Joint regression including an interaction term
reg_joint = smf.ols(formula = 'rprice ~ nearinc * C(year)', data = kielmc)
results_joint = reg_joint.fit()

In [9]:
# Print regression tables
table_78 = pd.DataFrame({'b': round(results78.params, 4),
                     'se': round(results78.bse, 4),
                     't': round(results78.tvalues, 4),
                     'pval': round(results78.pvalues, 4)})
print(f'Regression Table (1978): \n{table_78}\n')

table_81 = pd.DataFrame({'b': round(results81.params, 4),
                     'se': round(results81.bse, 4),
                     't': round(results81.tvalues, 4),
                     'pval': round(results81.pvalues, 4)})
print(f'Regression Table (1981): \n{table_81}\n')

table_joint = pd.DataFrame({'b': round(results_joint.params, 4),
                     'se': round(results_joint.bse, 4),
                     't': round(results_joint.tvalues, 4),
                     'pval': round(results_joint.pvalues, 4)})
print(f'Regression Table (Joint): \n{table_joint}\n')

Regression Table (1978): 
                    b        se        t    pval
Intercept  82517.2276  2653.790  31.0941  0.0000
nearinc   -18824.3705  4744.594  -3.9675  0.0001

Regression Table (1981): 
                     b         se        t  pval
Intercept  101307.5136  3093.0267  32.7535   0.0
nearinc    -30688.2738  5827.7088  -5.2659   0.0

Regression Table (Joint): 
                                  b         se        t    pval
Intercept                82517.2276  2726.9101  30.2603  0.0000
C(year)[T.1981]          18790.2860  4050.0650   4.6395  0.0000
nearinc                 -18824.3705  4875.3221  -3.8612  0.0001
nearinc:C(year)[T.1981] -11863.9033  7456.6462  -1.5911  0.1126



The **DID** estimator can be improved. A logarithmic specification is more plausible since it implies a constant percentage effect on the house values. We can also add additional regressors to control for incidental changes in the composition of the houses traded. In our example, we implement both improvements. The model including features of the houses implies an estimated decrease in the house values of about 13.2%. This effect is also significantly different from zero.

In [10]:
# Import modules
import wooldridge as woo
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf

In [11]:
# Import the 'kielmc' data set
kielmc = woo.dataWoo('kielmc')

In [12]:
# Difference-in-differences (DID)
reg_did = smf.ols(formula = 'np.log(rprice) ~ nearinc * C(year)', data = kielmc)
results_did = reg_did.fit()

In [13]:
# Print regression table
table_did = pd.DataFrame({'b': round(results_did.params, 4),
                     'se': round(results_did.bse, 4),
                     't': round(results_did.tvalues, 4),
                     'pval': round(results_did.pvalues, 4)})
print(f'Regression Table (DID): \n{table_did}\n')

Regression Table (DID): 
                               b      se         t    pval
Intercept                11.2854  0.0305  369.8386  0.0000
C(year)[T.1981]           0.1931  0.0453    4.2606  0.0000
nearinc                  -0.3399  0.0546   -6.2308  0.0000
nearinc:C(year)[T.1981]  -0.0626  0.0834   -0.7508  0.4533



In [14]:
# DID with control variables
reg_didC = smf.ols(formula = 'np.log(rprice) ~ nearinc*C(year) + age +'
                  'I(age**2) + np.log(intst) + np.log(land) + np.log(area) +'
                  'rooms + baths', data = kielmc)
results_didC = reg_didC.fit()

In [15]:
# Print regression table
table_didC = pd.DataFrame({'b': round(results_didC.params, 4),
                     'se': round(results_didC.bse, 4),
                     't': round(results_didC.tvalues, 4),
                     'pval': round(results_didC.pvalues, 4)})
print(f'Regression Table (DID w/ Control Variables): \n{table_didC}\n')

Regression Table (DID w/ Control Variables): 
                              b      se        t    pval
Intercept                7.6517  0.4159  18.3986  0.0000
C(year)[T.1981]          0.1621  0.0285   5.6868  0.0000
nearinc                  0.0322  0.0475   0.6789  0.4977
nearinc:C(year)[T.1981] -0.1315  0.0520  -2.5305  0.0119
age                     -0.0084  0.0014  -5.9236  0.0000
I(age ** 2)              0.0000  0.0000   4.3415  0.0000
np.log(intst)           -0.0614  0.0315  -1.9500  0.0521
np.log(land)             0.0998  0.0245   4.0766  0.0001
np.log(area)             0.3508  0.0515   6.8129  0.0000
rooms                    0.0473  0.0173   2.7317  0.0067
baths                    0.0943  0.0277   3.4003  0.0008



### 3. Organizing Panel Data

A panel data set includes several observations at different points in time *t* for the same (or at least an overlapping) set of cross-section units *i*. A simple "pooled" regression model could look like

$$y_{it} = \beta_0 + \beta_1 x_{it1} + \beta_2 x_{it2} + ... + \beta_k x_{itk} + u_{it}$$

$$t = 1, 2, 3, ..., T$$

$$i = 1, 2, 3, ..., n$$

where the double subscript now indicate values for individual (or other cross-sectional unit) *i* at tie *t*. We could estimate this model by OLS, essentially ignoring the panel structure. But at least the assumption that the error terms are unrelated is very hard to justify since they contain unobserved individual traits that are likely to be constant or at least correlated over time. Therefore, we need specific methods for panel data.

For the caluculations used by panel data methods, we have to make sure that the data set is systematically organized and the estimation routines understand its structure. Usually, a panel data set comes in a "long" form where each row of data corresponds to one combination of *i* and *t*. We have to define which observations belong together by introducing an index variable for the cross-sectional units *i* and preferably also the time index *t*.

The module **linearmodels** is a comprehesive collection of commands dealing with panel data. It is not part of the Anaconda distribution and you have to install it. When working with panel data in **linearmodels**, our first line of code always is:

``` Python
import linearmodels as plm
```

The routines require a **Pandas** data frame with a two-dimensional index that describe the individual and time dimenisions. Suppose we have our data in a standard data frame named **mydf**. It includes a variable **ivar** indicating the cross-sectional units and a variable **tvar** indicating the time. To work with **linearmodels** we create a data frame with the command

``` Python
mydf = mydf.set_index(['ivar', 'tvar'])
```

Let's apply this to the data set *CRIME2* discussed by Wooldridge (2019, SEction 13.3). It is a balanced panel of 46 cities, properly sorted. We first import the data set and set the indices correctly.

Once we use routines from **linearmodel**, it will report the number of cross-section units *n*, the number of time units *T*, and the total number of observations *N*. 

In [16]:
# Import modules
import wooldridge as woo
import pandas as pd
import linearmodels as plm

In [17]:
# Import the 'crime2' data set
crime2 = woo.dataWoo('crime2')
crime2.head()

,pop,crimes,unem,officers,pcinc,west,nrtheast,south,year,area,...,clcrimes,clpop,clcrmrte,lpolpc,clpolpc,cllawexp,cunem,clpopden,lcrmrt_1,ccrmrte
0,229528.0,17136.0,8.2,326,8532,1,0,0,82,44.599998,...,NaN,NaN,NaN,0.350872,NaN,NaN,NaN,NaN,NaN,NaN
1,246815.0,17306.0,3.7,321,12155,1,0,0,87,44.599998,...,0.009871,0.072614,-0.062743,0.262802,-0.08807,0.977952,-4.5,0.072615,4.312912,-4.540268
2,814054.0,75654.0,8.1,1621,7551,1,0,0,82,375.000000,...,NaN,NaN,NaN,0.688772,NaN,NaN,NaN,NaN,NaN,NaN
3,933177.0,83960.0,5.4,1803,11363,1,0,0,87,375.000000,...,0.104170,0.136568,-0.032398,0.658612,-0.03016,0.200762,-2.7,0.136568,4.531899,-2.962654
4,374974.0,31352.0,9.0,633,8343,1,0,0,82,49.799999,...,NaN,NaN,NaN,0.523614,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# Create time variable dummy by converting a Boolean variable to an interger
crime2['t'] = (crime2['year'] == 87).astype(int)

In [19]:
# Create an index in this balanced data set by combining two arrays
id_tmp = np.linspace(1, 46, num = 46)
crime2['id'] = np.sort(np.concatenate([id_tmp, id_tmp]))

In [20]:
# Create new dataframe with indices 'id' and 'year'
crime_new = crime2.set_index(['id', 'year'])
crime_new.head()

pop   crimes  unem  officers  pcinc  west  nrtheast  south  \
id  year                                                                    
1.0 82    229528.0  17136.0   8.2       326   8532     1         0      0   
    87    246815.0  17306.0   3.7       321  12155     1         0      0   
2.0 82    814054.0  75654.0   8.1      1621   7551     1         0      0   
    87    933177.0  83960.0   5.4      1803  11363     1         0      0   
3.0 82    374974.0  31352.0   9.0       633   8343     1         0      0   

                area  d87  ...     clpop  clcrmrte    lpolpc  clpolpc  \
id  year                   ...                                          
1.0 82     44.599998    0  ...       NaN       NaN  0.350872      NaN   
    87     44.599998    1  ...  0.072614 -0.062743  0.262802 -0.08807   
2.0 82    375.000000    0  ...       NaN       NaN  0.688772      NaN   
    87    375.000000    1  ...  0.136568 -0.032398  0.658612 -0.03016   
3.0 82     49.799999    0  ...       NaN       NaN  0.523614      NaN   

          cllawexp  cunem  clpopden  lcrmrt_1   ccrmrte  t  
id  year                                                    
1.0 82         NaN    NaN       NaN       NaN       NaN  0  
    87    0.977952   -4.5  0.072615  4.312912 -4.540268  1  
2.0 82         NaN    NaN       NaN       NaN       NaN  0  
    87    0.200762   -2.7  0.136568  4.531899 -2.962654  1  
3.0 82         NaN    NaN       NaN       NaN       NaN  0  

[5 rows x 34 columns]

### 4. First Differenced Estimator

Wooldridge (2019, SEctions 13.3 - 13.5) discusses basic unobserved effects models and their estimation by **first-differencing (FD)**. Consider the model

$$y_{it} = \beta_0 + \beta_1 x_{it1} + \beta_2 x_{it2} + ... + \beta_k x_{itk} + a_i + u_{it}$$

$$t = 1, 2, 3, ..., T$$

$$i = 1, 2, 3, ..., n$$

which explicitly involves an unobserved effect $a_i$ that is constant over time (since it has no *t* subscript). If it is correlated with one or more of the regressors $x_{it1}, x_{it2}, ..., x_{itk}$ we cannot simply ignore $a_i$, leave it in the composite error term $u_{it} = a_i + u_{it}$ and estimate the equation by OLS. The error term $u_{it}$ would be related to the regressors, violating the assumption MLR.4 and creating biases and inconsistencies. Note that this problem is not unique to panel data, but possible solutions are.

The **first differenced (FD)** estimator is based on the first difference of the whole equation

$$\Delta y_{it} \equiv y_{it} - y_{it-1}$$

$$= \beta_1 \Delta x_{it1} + \beta_2 \Delta x_{it2} + ... + \beta_k \Delta x_{itk} + \Delta u_{it}$$

$$t = 2, 3, 4, ..., T$$

$$i = 1, 2, 3, ..., n$$

Note that we cannot evaluate this equation for the first observation *t* = 1 for any *i* since the lagged values are unknown for them. The trick is that $a_i$ drops out of the euqation by differencing since it does not change over time. No matter how badly it is correlated with the regressors, it cannot hurt the estimation anymore. This estimating equation is then analyzed by OLS. We simply regress the differenced dependent variable $\Delta y_{it}$ on the differenced independent variables $\Delta x_{it1}, \Delta x_{it2}, ..., \Delta x_{itk}$

We describe the cumbersome data preparation required fro the manual estimation. Before we can use the method **diff()** to calculate first differences of the dependent variable crime rate (**crmrte**) and the independent variable unemployment rate (**unem**), we have to maek sure with **groupby('id')** that these calculations are performed per individual.

A list of the first five observations reveals that the differences are unavailable (**NaN**) for the first year of each city. The other differences are also calculated as expected. For example the change fo the crime rate for city 1 is 70.11729 - 74.65756 = -4.540268 and the change fo the unemployment rate for city 2 is 5.4 - 8.1 = -2.7. The **FD** estimator can now be calculated by simply applying OLS to these differenced values. The observations for the first year with missing information are automatically dropped from the estimation sample. The results show a significantly positive relation between unemployement and crime.

In [21]:
# Import modules
import wooldridge as woo
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import linearmodels as plm

In [22]:
# Import the 'crime2' data set
crime2 = woo.dataWoo('crime2')
crime2.head()

,pop,crimes,unem,officers,pcinc,west,nrtheast,south,year,area,...,clcrimes,clpop,clcrmrte,lpolpc,clpolpc,cllawexp,cunem,clpopden,lcrmrt_1,ccrmrte
0,229528.0,17136.0,8.2,326,8532,1,0,0,82,44.599998,...,NaN,NaN,NaN,0.350872,NaN,NaN,NaN,NaN,NaN,NaN
1,246815.0,17306.0,3.7,321,12155,1,0,0,87,44.599998,...,0.009871,0.072614,-0.062743,0.262802,-0.08807,0.977952,-4.5,0.072615,4.312912,-4.540268
2,814054.0,75654.0,8.1,1621,7551,1,0,0,82,375.000000,...,NaN,NaN,NaN,0.688772,NaN,NaN,NaN,NaN,NaN,NaN
3,933177.0,83960.0,5.4,1803,11363,1,0,0,87,375.000000,...,0.104170,0.136568,-0.032398,0.658612,-0.03016,0.200762,-2.7,0.136568,4.531899,-2.962654
4,374974.0,31352.0,9.0,633,8343,1,0,0,82,49.799999,...,NaN,NaN,NaN,0.523614,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
# Create time variable dummy by converting a Boolean variable to an interger
crime2['t'] = (crime2['year'] == 87).astype(int)

In [24]:
# Create an index in this balanced data set by combining two arrays
id_tmp = np.linspace(1, 46, num = 46)
crime2['id'] = np.sort(np.concatenate([id_tmp, id_tmp]))

In [25]:
# Manually calculate first difference per entity for crmrte and unem
crime2['crmrte_diff1'] = crime2.sort_values(['id', 'year']).groupby('id')['crmrte'].diff()

crime2['unem_diff1'] = crime2.sort_values(['id', 'year']).groupby('id')['unem'].diff()

var_selection = ['id', 't', 'crimes', 'unem', 'crmrte_diff1', 'unem_diff1']

print(f'Data Frame with Selected Variables: \n{crime2[var_selection].head()}\n')

Data Frame with Selected Variables: 
    id  t   crimes  unem  crmrte_diff1  unem_diff1
0  1.0  0  17136.0   8.2           NaN         NaN
1  1.0  1  17306.0   3.7     -4.540268        -4.5
2  2.0  0  75654.0   8.1           NaN         NaN
3  2.0  1  83960.0   5.4     -2.962654        -2.7
4  3.0  0  31352.0   9.0           NaN         NaN



In [26]:
# Estimate FD model with statsmodels on differenced data
reg_sm = smf.ols(formula = 'crmrte_diff1 ~ unem_diff1', data = crime2)
results_sm = reg_sm.fit()

In [27]:
# Print regression table
table_sm = pd.DataFrame({'b': round(results_sm.params, 4),
                     'se': round(results_sm.bse, 4),
                     't': round(results_sm.tvalues, 4),
                     'pval': round(results_sm.pvalues, 4)})
print(f'Regression Table (StatsModels): \n{table_sm}\n')

Regression Table (StatsModels): 
                  b      se       t    pval
Intercept   15.4022  4.7021  3.2756  0.0021
unem_diff1   2.2180  0.8779  2.5266  0.0152



In [28]:
# Estimate FD model with linearmodels
crime2 = crime2.set_index(['id', 'year'])
reg_plm = plm.FirstDifferenceOLS.from_formula(formula = 'crmrte ~ t + unem',
                                             data = crime2)
results_plm = reg_plm.fit()

In [29]:
# Print regression table
table_plm = pd.DataFrame({'b': round(results_plm.params, 4),
                     'se': round(results_plm.std_errors, 4),
                     't': round(results_plm.tstats, 4),
                     'pval': round(results_plm.pvalues, 4)})
print(f'Regression Table (LinearModels): \n{table_plm}\n')

Regression Table (LinearModels): 
            b      se       t    pval
t     15.4022  4.7021  3.2756  0.0021
unem   2.2180  0.8779  2.5266  0.0152



Generating the differenced values and using **ols** on them is actually unnecessary. The command **FirstDifferenceOLS** shows that many lines of code can be saved by using the canned routine in **linearmodels**. All the necessary calculations are done internally. As the output shows, the parameters are therefore exactly the same as our pedestrian calculations.

#### Wooldridge, Example 13.9: County Crime Rates in North Carolina

In this example, we analyze the data *CRIME4*. We estimate the model in first differences using **linearmodels**.

Note that in this sepcification, all variables are automatically differenced, so they have the intuitive interpretation in the level equation. In the results reported by Wooldridge (2019), the year dummies are not differenced which only makes a difference for the interpretation of the year coefficients. We will repeat this example with "robust" standard errors.

In [30]:
# Import modules
import wooldridge as woo
import numpy as np
import linearmodels as plm

In [31]:
# Import the 'crime2' data set
crime4 = woo.dataWoo('crime4')

# Create new indices with county and year
crime4 = crime4.set_index(['county', 'year'], drop = False)

In [32]:
# Estimate FD model
reg = plm.FirstDifferenceOLS.from_formula(
    formula = 'np.log(crmrte) ~ year + d83 + d84 + d85 + d86 + d87 +'
                'lprbarr + lprbconv + lprbpris + lavgsen + lpolpc',
            data = crime4)
results = reg.fit()

In [33]:
# Print regression table
print(f'First Difference OLS Regression: \n {results} \n')

First Difference OLS Regression: 
                      FirstDifferenceOLS Estimation Summary                      
Dep. Variable:         np.log(crmrte)   R-squared:                        0.4326
Estimator:         FirstDifferenceOLS   R-squared (Between):              0.6003
No. Observations:                 540   R-squared (Within):               0.4281
Date:                Fri, Jul 09 2021   R-squared (Overall):              0.6000
Time:                        22:46:20   Log-likelihood                    248.48
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      36.661
Entities:                          90   P-value                           0.0000
Avg Obs:                       7.0000   Distribution:                  F(11,529)
Min Obs:                       7.0000                                           
Max Obs:                       7.0000   F-statistic (robust):             